# Rotor Estimation using the Tensor Representation of Geometric Algebra

In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=2, suppress=True,threshold=np.inf)
import versor as vsr

Populating the interactive namespace from numpy and matplotlib


## Dataset generation

In [2]:
r = vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)

In [4]:
R = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        a = vsr.Vec(0,0,0)
        b = vsr.Vec(0,0,0)
        a[j] = 1
        b[i] = 1
        R[i,j] = b <= a.spin(r)        

In [5]:
R

array([[ 0.5 ,  0.  , -0.87],
       [ 0.  ,  1.  ,  0.  ],
       [ 0.87,  0.  ,  0.5 ]])

In [6]:
vsr.Vec(1,0,0).spin(r)

Vec: [ 0.5 0 0.87 ]

In [7]:
vsr.Vec(0,1,0).spin(r)

Vec: [ 0 1 0 ]

In [8]:
vsr.Vec(0,0,1).spin(r)

Vec: [ -0.87 0 0.5 ]

In [9]:
n_points = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3))
            for i in range(n_points)]
points_b = [point.spin(rotor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))) 
                  for point in points_b]

NameError: name 'rotor' is not defined

In [36]:
rotor = vsr.Biv(0,-pi/8,0).exp()
print(rotor)
n_points = 3
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3))
            for i in range(n_points)]
points_b = [point.spin(rotor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))) 
                  for point in points_b]

ega_a = [vsr.EGA(p) for p in points_a]
ega_b = [vsr.EGA(p) for p in points_b]

M = np.array([[1,0,0,0,0,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,1,0]])
print(M)

def matrix(a, b):
    m = np.zeros((8,8))
    for i in range(8):
        ei = vsr.EGA(0,0,0,0,0,0,0,0)
        ei[i] = i
        m[i,:] = np.array(ei * a - b * ei)
    return m


# m = np.row_stack([
#         np.delete(np.delete(matrix(a,b),[0,4,5,6],0), [1,2,3,7],1)
#         for a, b in zip(ega_a, ega_b)]).copy()

m = np.row_stack(np.dot(matrix(a,b),M.T) for a, b in zip(ega_a, ega_b))

U,s,Vt = np.linalg.svd(m)
# print(Vt.T)
print(vsr.Rot(*Vt[-1]))
# print(s)

Rot: [ 0.92 0 0.38 0 ]
[[1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]]
Rot: [ -0.92 5.6e-17 -0.38 -6.9e-18 ]


## Solver

In [37]:
class Multivector(object):
    def __init__(self, data=None):
        if data is not None:
            self._data = np.array(data).reshape(8,1)
        else:
            self._data = np.zeros((8,1))
        self._grades = [0, 1, 1, 1, 2, 2, 2, 3]
        self._gp_tensor = self._create_gp_tensor()
        
    def __repr__(self):
        return self._data.ravel().__repr__()
    
#     @property
#     def scalar(self):
#         return self._data[0]
    
#     @scalar.setter
    def scalar(self, scalar):
        self._data[0] = scalar
        return Multivector(self._data)
    
#     @property        
        gp_table = np.array([1,  2,  3,  4,  5,  6,  7,  8,
                             2,  1,  7, -6,  8, -4,  3,  5,
                             3, -7,  1,  5,  4,  8, -2,  6,
                             4,  6, -5,  1, -3,  2,  8,  7,
                             5,  8, -4,  3, -1, -7,  6, -2,
                             6,  4,  8, -2,  7, -1, -5, -3,
                             7, -3,  2,  8, -6,  5, -1, -4, 
                             8,  5,  6,  7, -2, -3, -4, -1]).T.reshape(8,8)
#     def vector(self):
#         return self._data[1:4]
    
    def vector(self, vector):
        self._data[1:4] = np.array(vector).copy().reshape(-1,1)
        return Multivector(self._data)
    
#     @property
#     def bivector(self):
#         return self._data[4:7]
    
#     @bivector.setter
    def bivector(self, bivector):
        self._data[4:7] = np.array(bivector).copy().reshape(-1,1)
        return Multivector(self._data)
    
#     @property
#     def pseudoscalar(self):
#         return self._data[7]
    
#     @pseudoscalar.setter
    def pseudoscalar(self, pseudoscalar):
        self._data[7] = pseudoscalar
        return Multivector(self._data)
    
    def _create_gp_tensor(self):
        M = np.zeros((8,8))
        mask = np.array([1,2,3,4,5,6,7,8])
        for i in range(8):
            W = np.zeros((8,8))
            for j in range(8):
                a = vsr.EGA(0,0,0,0,0,0,0,0)
                b = vsr.EGA(0,0,0,0,0,0,0,0)
                a[i] = 1.
                b[j] = 1.
                M[i,j] = np.dot(mask, np.array(a * b))
        gp_table = M.copy()
        tensor = np.zeros((8,8,8))
        for k in range(8):
            for i in range(8):
                for j in range(8):
                    val = gp_table[i,j]
                    if abs(val) == k + 1:
                        tensor[k,i,j] = np.sign(val)
        return tensor
    
    def gp_right_matrix(self):
        return np.tensordot(self._gp_tensor.T,self._data,1).reshape(8,8)
    
    def gp_left_matrix(self):
        return np.tensordot(self._data.T, self._gp_tensor,1).reshape(8,8)

In [38]:
Multivector(vsr.EGA(vsr.Vec(1,2,3))).gp_left_matrix()

array([[ 0.,  1.,  2.,  3.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  2.,  3.,  0.,  0.],
       [ 2.,  0.,  0.,  0., -1.,  0.,  3.,  0.],
       [ 3.,  0.,  0.,  0.,  0., -1., -2.,  0.],
       [ 0., -2.,  1.,  0.,  0.,  0.,  0., -3.],
       [ 0., -3.,  0.,  1.,  0.,  0.,  0.,  2.],
       [ 0.,  0., -3.,  2.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -3.,  2., -1.,  0.]])

In [39]:
matrix(vsr.EGA(vsr.Vec(1,2,3)))

TypeError: matrix() takes exactly 2 arguments (1 given)

In [40]:
np.dot(Multivector(vsr.EGA(vsr.Vec(1,2,3))).gp_left_matrix(), vsr.EGA(vsr.Vec(-5,-5,-7)))

array([-36.,   0.,   0.,   0.,   5.,   8.,   1.,   0.])

In [41]:
vsr.Vec(1,2,3) * vsr.Vec(-5,-5,-7)

Rot: [ -36 5 8 1 ]

In [42]:
vsr.Vec(-5,-5,-7) * vsr.Vec(1,2,3)

Rot: [ -36 -5 -8 -1 ]

In [43]:
def matrix(a, left=True):
    m = np.zeros((8,8))
    for i in range(8):
        ei = vsr.EGA(0,0,0,0,0,0,0,0)
        ei[i] = 1.0
        if left:
            m[i,:] = np.array(a * ei)
        else:
            m[i,:] = np.array(ei * a)
    return m

In [44]:
mask = [1,0,0,0,1,1,1,0]
mask= np.outer(mask,mask)
m = matrix(vsr.EGA(vsr.Vec(1,2,3))) - matrix(vsr.EGA(vsr.Vec(3,-1,5)),True) 
print(m)
np.delete(np.delete(m,[0,4,5,6],0), [1,2,3,7],1)

[[ 0. -2.  3. -2.  0.  0.  0.  0.]
 [-2.  0.  0.  0. -3.  2.  0.  0.]
 [ 3.  0.  0.  0. -2.  0.  2.  0.]
 [-2.  0.  0.  0.  0. -2.  3.  0.]
 [ 0. -3. -2.  0.  0.  0.  0. -2.]
 [ 0.  2.  0. -2.  0.  0.  0. -3.]
 [ 0.  0.  2.  3.  0.  0.  0. -2.]
 [ 0.  0.  0.  0. -2. -3. -2.  0.]]


array([[-2., -3.,  2.,  0.],
       [ 3., -2.,  0.,  2.],
       [-2.,  0., -2.,  3.],
       [ 0., -2., -3., -2.]])

In [45]:
motor

NameError: name 'motor' is not defined

In [46]:
points_a = [vsr.EGA(vsr.Vec(1,0,0)), 
            vsr.EGA(vsr.Vec(0,0,1)),
            vsr.EGA(vsr.Vec(1,2,3))]
points_b = [a.spin(vsr.EGA(rotor)) for a in points_a]

# n_points = 10
# sigma = 0.09
# points_a = [vsr.EGA(vsr.Vec(*np.random.normal(0.0, 0.8, 3)))
#             for i in range(n_points)]
# points_b = [point.spin(vsr.EGA(rotor)) for point in points_a]

m = np.array([gp_a - gp_b for gp_a, gp_b in zip([Multivector(np.array(point)).gp_right_matrix()
                                                 for point in points_a],
                                                [Multivector(np.array(point)).gp_left_matrix()
                                                 for point in points_b])]).reshape(-1,8)
U,s,Vt = np.linalg.svd(m)
print(s)
print(Vt.T)
print(rotor)

[ 7.93  7.93  7.75  7.75  2.26  2.26  0.    0.  ]
[[-0.   -0.33  0.   -0.    0.    0.19  0.48  0.79]
 [ 0.33  0.   -0.72 -0.24  0.56  0.   -0.   -0.  ]
 [-0.8   0.    0.    0.    0.47  0.   -0.33  0.2 ]
 [ 0.38  0.    0.62  0.21  0.65  0.   -0.   -0.  ]
 [-0.   -0.38 -0.21  0.62  0.   -0.65  0.    0.  ]
 [ 0.   -0.8  -0.    0.   -0.    0.47 -0.2  -0.33]
 [ 0.   -0.33  0.24 -0.72 -0.   -0.56 -0.    0.  ]
 [-0.33 -0.    0.    0.    0.19 -0.    0.79 -0.48]]
Rot: [ 0.92 0 0.38 0 ]


In [47]:
Multivector().vector(points_a[0]).gp_left_matrix()

ValueError: could not broadcast input array from shape (8,1) into shape (3,1)

In [48]:
class TensorRotorSolver(object):
    def __init__(self):
        self._gp_tensor = self._create_gp_tensor()
        
    @property
    def gp_tensor(self):
        return self._gp_tensor
        
    def _create_gp_tensor(self):
        gp_table = np.array([1,  2,  3,  4,  5,  6,  7,  8,
                             2,  1,  7, -6,  8, -4,  3,  5,
                             3, -7,  1,  5,  4,  8, -2,  6,
                             4,  6, -5,  1, -3,  2,  8,  7,
                             5,  8, -4,  3, -1, -7,  6, -2,
                             6,  4,  8, -2,  7, -1, -5, -3,
                             7, -3,  2,  8, -6,  5, -1, -4, 
                             8,  5,  6,  7, -2, -3, -4, -1]).T.reshape(8,8)
        tensor = np.zeros((8,8,8))
        for k in range(8):
            for i in range(8):
                for j in range(8):
                    val = gp_table[i,j]
                    if abs(val) == k + 1:
                        tensor[k,i,j] = np.sign(val)
        return tensor
    
    

In [49]:
Gkij = TensorRotorSolver().gp_tensor
ai = np.array([0,1,2,3,0,0,0,0])
bj = np.array([0,0,0,0,1,2,3,0])
print(np.einsum('i,j,kij->k', ai, bj, Gkij))

[  0.   0.   0.   0.   0.   0.   0.  14.]


In [50]:
print(np.einsum('j,kij->ki',bj, Ikij))

NameError: name 'Ikij' is not defined

In [51]:
print(np.einsum('i,kij->kj', ai, Gkij))

[[ 0.  1.  2.  3.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  3. -2.  0.]
 [ 2.  0.  0.  0. -3.  0.  1.  0.]
 [ 3.  0.  0.  0.  2. -1.  0.  0.]
 [ 0.  0. -3.  2.  0.  0.  0.  1.]
 [ 0.  3.  0. -1.  0.  0.  0.  2.]
 [ 0. -2.  1.  0.  0.  0.  0.  3.]
 [ 0.  0.  0.  0.  1.  2.  3.  0.]]


In [52]:
vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(0,1,2,3,4,5,6,7)

EGA: [ 1 0 4 5 2 3 7 6 ]

In [53]:
B = vsr.EGA(0,0,0,0,5,6,7,0)
J = np.zeros((8,8))
for i in range(8):
    ei = vsr.EGA(*np.zeros(8))
    ei[i] = 1.
    J[:,i] = ei <= B
print(J)
    

[[ 0.  0.  0.  0. -5. -6. -7.  0.]
 [ 0.  0. -5. -6.  0.  0.  0.  0.]
 [ 0.  5.  0. -7.  0.  0.  0.  0.]
 [ 0.  6.  7.  0.  0.  0.  0.  0.]
 [ 5.  0.  0.  0.  0.  0.  0.  0.]
 [ 6.  0.  0.  0.  0.  0.  0.  0.]
 [ 7.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


In [54]:
print(np.einsum('i,j,kij->k', ai, bj, Ikij))

NameError: name 'Ikij' is not defined

In [55]:
print(np.einsum('i,j,kij->k', ai, bj, Okij))

NameError: name 'Okij' is not defined

In [56]:
vsr.EGA(1,2,3,4,5,6,7,8).rev()

EGA: [ 1 2 3 4 -5 -6 -7 -8 ]

In [57]:
Rji = np.array([1,0,0,0,0,0,0,0,
           0,1,0,0,0,0,0,0,
          0,0,1,0,0,0,0,0,
          0,0,0,1,0,0,0,0,
          0,0,0,0,-1,0,0,0,
          0,0,0,0,0,-1,0,0,
          0,0,0,0,0,0,-1,0,
          0,0,0,0,0,0,0,-1
         ]).reshape(8,8)

In [58]:
rot = np.array([cos(pi/6),0,0,0,0,0,-sin(pi/6),0])
rotrev = np.einsum('i,ji->j', rot, Rji)
print(rot, rotrev)

[ 0.87  0.    0.    0.    0.    0.   -0.5   0.  ] [ 0.87  0.    0.    0.    0.    0.    0.5   0.  ]


In [59]:
print(np.einsum('i,j,m,kij,ml,pkl->p', rot, ai, rot,Gkij,Rji, Gkij))

[ 0.   -1.23  1.87  3.    0.    0.    0.    0.  ]


In [60]:
print(np.einsum('i,j,m,lm,kij,pkl->p', rot, ai, rot,Rji,Gkij,Gkij))

[ 0.   -1.23  1.87  3.    0.    0.    0.    0.  ]


In [61]:
print(np.einsum('j,m,ml,pkl->p', ai, rot,Rji,Gkij,Gkij))

ValueError: fewer operands provided to einstein sum function than specified in the subscripts string

In [62]:
print(np.einsum('j,m,ml,kij,pkl->pi', ai, rot,Rji,Gkij,Gkij) + 
      np.einsum('i,j,kij,pkl->pl', rot, ai, Gkij,Gkij))

[[ 0.   -0.27  4.46  5.2   0.    0.    0.    0.  ]
 [-0.27  0.    0.    0.    0.    0.    0.    0.  ]
 [ 4.46  0.    0.    0.    0.    0.    0.    0.  ]
 [ 5.2   0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.   -0.27]
 [ 0.    0.    0.    0.    0.    0.    0.    4.46]
 [ 0.    0.    0.    0.    0.    0.    0.    5.2 ]
 [ 0.    0.    0.    0.   -0.27  4.46  5.2   0.  ]]


In [63]:
print(np.einsum('i,j,lm,kij,pkl->pm', rot, ai,Rji,Gkij,Gkij) +
      np.einsum('j,m,lm,kij,pkl->pi', ai, rot,Rji,Gkij,Gkij))

[[ 0.   -0.27  4.46  5.2   0.    0.    0.   -3.  ]
 [-0.27  0.    0.    0.   -3.   -5.2   4.46  0.  ]
 [ 4.46  0.    0.    0.    5.2  -3.    0.27  0.  ]
 [ 5.2   0.    0.    0.   -4.46 -0.27 -3.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]]


In [64]:
print(np.einsum('j,m,lm,kij,pkl->ip', ai, rot,Rji,Gkij,Gkij))

[[ 0.   -0.13  2.23  2.6   0.    0.    0.    1.5 ]
 [-0.13  0.    0.    0.    1.5  -2.6   2.23  0.  ]
 [ 2.23  0.    0.    0.    2.6   1.5   0.13  0.  ]
 [ 2.6   0.    0.    0.   -2.23 -0.13  1.5   0.  ]
 [ 0.   -1.5   2.6  -2.23  0.    0.    0.   -0.13]
 [ 0.   -2.6  -1.5  -0.13  0.    0.    0.    2.23]
 [ 0.    2.23  0.13 -1.5   0.    0.    0.    2.6 ]
 [-1.5   0.    0.    0.   -0.13  2.23  2.6   0.  ]]


In [65]:
np.einsum('r,j,kij->')

ValueError: must specify the einstein sum subscripts string and at least one operand

In [66]:
Jac = np.zeros((3,4))
Jac[:,0] = np.array(vsr.EGA(1,0,0,0,0,0,0,0) * ae * Re.rev() + Re * ae * vsr.EGA(1,0,0,0,0,0,0,0))[1:4]
Jac[:,1] = np.array(vsr.EGA(0,0,0,0,1,0,0,0) * ae * Re.rev() + Re * ae * vsr.EGA(0,0,0,0,-1.,0,0,0))[1:4]
Jac[:,2] = np.array(vsr.EGA(0,0,0,0,0,1,0,0) * ae * Re.rev() + Re * ae * vsr.EGA(0,0,0,0,0,-1,0,0))[1:4]
Jac[:,3] = np.array(vsr.EGA(0,0,0,0,0,0,1.,0) * ae * Re.rev() + Re * ae * vsr.EGA(0,0,0,0,0,0,-1,0))[1:4]
print(Jac)

NameError: name 'ae' is not defined

In [67]:
ae = vsr.EGA(0,1,0,0,0,0,0,0)
Re = vsr.EGA(cos(pi/6),0,0,0,-sin(pi/6),0,0,0)
Jac = np.zeros((8,8))
Jac[:,0] = np.array(vsr.EGA(1,0,0,0,0,0,0,0) * ae * Re.rev() + Re * ae * vsr.EGA(1,0,0,0,0,0,0,0))
Jac[:,1] = np.array(vsr.EGA(0,0,0,0,1,0,0,0) * ae * Re.rev() + Re * ae * vsr.EGA(0,0,0,0,-1.,0,0,0))
Jac[:,2] = np.array(vsr.EGA(0,0,0,0,0,1,0,0) * ae * Re.rev() + Re * ae * vsr.EGA(0,0,0,0,0,-1,0,0))
Jac[:,3] = np.array(vsr.EGA(0,0,0,0,0,0,1.,0) * ae * Re.rev() + Re * ae * vsr.EGA(0,0,0,0,0,0,-1,0))
print(Jac)

[[ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 1.73  1.    0.    0.    0.    0.    0.    0.  ]
 [ 1.   -1.73  0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.   -1.73 -1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]]


In [68]:
vsr.Vec(1,0,0).spin(vsr.Rot(cos(pi/6), -sin(pi/6),0,0))

Vec: [ 0.5 0.87 0 ]

In [69]:
def create_ip_tensor():
    gp_table = np.array([0, 0, 0, 0, 0, 0, 0, 0, 
                         0, 1, 0, 0, 0, -4, 3, 5, 
                         0, 0, 1, 0, 4, 0, -2, 6, 
                         0, 0, 0, 1, -3, 2, 0, 7, 
                         0, 0, -4, 3, -1, 0, 0, -2, 
                         0, 4, 0, -2, 0, -1, 0, -3, 
                         0, -3, 2, 0, 0, 0, -1, -4, 
                         0, 5, 6, 7, -2, -3, -4, -1]).T.reshape(8,8)
    tensor = np.zeros((8,8,8))
    for k in range(8):
        for i in range(8):
            for j in range(8):
                val = gp_table[i,j]
                if abs(val) == k + 1:
                    tensor[k,i,j] = np.sign(val)
    return tensor

def create_op_tensor():
    gp_table = np.array([1, 2, 3, 4, 5, 6, 7, 8,
                         2, 0, 7, -6, 8, 0, 0, 0,
                         3, -7, 0, 5, 0, 8, 0, 0, 
                         4, 6, -5, 0, 0, 0, 8, 0, 
                         5, 8, 0, 0, 0, 0, 0, 0,
                         6, 0, 8, 0, 0, 0, 0, 0, 
                         7, 0, 0, 8, 0, 0, 0, 0, 
                         8, 0, 0, 0, 0, 0, 0, 0]).T.reshape(8,8)
    tensor = np.zeros((8,8,8))
    for k in range(8):
        for i in range(8):
            for j in range(8):
                val = gp_table[i,j]
                if abs(val) == k + 1:
                    tensor[k,i,j] = np.sign(val)
    return tensor


In [70]:
Ikij = create_ip_tensor()
Okij = create_op_tensor()

In [71]:
BjIkij = np.einsum('j,kij->ki',B, Ikij)
print(BjIkij)

[[ 0.  0.  0.  0. -5. -6. -7.  0.]
 [ 0.  0. -7.  6.  0.  0.  0. -5.]
 [ 0.  7.  0. -5.  0.  0.  0. -6.]
 [ 0. -6.  5.  0.  0.  0.  0. -7.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


In [72]:
np.tensordot(a, BjIkij,1)

array([  0.,  -4.,   8.,  -4.,   0.,   0.,   0., -38.])

In [73]:
np.einsum('j,ijk->ki',B, Gkij)

array([[ 0.,  0.,  0.,  0.,  5.,  6.,  7.,  0.],
       [ 0.,  0., -7.,  6.,  0.,  0.,  0.,  5.],
       [ 0.,  7.,  0., -5.,  0.,  0.,  0.,  6.],
       [ 0., -6.,  5.,  0.,  0.,  0.,  0.,  7.],
       [-5.,  0.,  0.,  0.,  0., -7.,  6.,  0.],
       [-6.,  0.,  0.,  0.,  7.,  0., -5.,  0.],
       [-7.,  0.,  0.,  0., -6.,  5.,  0.,  0.],
       [ 0., -5., -6., -7.,  0.,  0.,  0.,  0.]])

In [74]:
Gkij = np.zeros((4,4,4))
Gkij[0] = np.array([1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1]).reshape(4,4)
Gkij[1] = np.array([0,1,0,0,1,0,0,0,0,0,0,-1,0,0,1,0]).reshape(4,4)
Gkij[2] = np.array([0,0,1,0,0,0,0,1,1,0,0,0,0,-1,0,0]).reshape(4,4)
Gkij[3] = np.array([0,0,0,1,0,0,1,0,0,-1,0,0,1,0,0,0]).reshape(4,4)
Gkij

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0., -1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.]],

       [[ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.],
        [ 1.,  0.,  0.,  0.]]])

In [75]:
ai = np.array([0,1,0,0])
bj = np.array([0,0,1,0])
np.einsum('i,j,kij->k',ai,bj,Gkij)

array([ 0.,  0.,  0.,  1.])

In [76]:
# Reduced tensor

In [77]:
Maji = Mbji = np.array([[0,1,0,0],[0,0,1,0]])
Mcji = np.array([[1,0,0,0],[0,0,0,1]])

In [78]:
Gwuv = np.einsum('wk,iu,jv,kij->wuv',Mcji,Maji.T, Mbji.T,Gkij)

In [79]:
aM = np.array([0,1]) # 0 e1 + 1 e2
bM = np.array([1,0]) # 1 e1 + 0 e2

In [80]:
cM = np.einsum('u,v,wuv->w',aM, bM, Gwuv)

In [81]:
np.einsum('w,wk',cM, Mcji)

array([ 0.,  0.,  0., -1.])

In [82]:
np.tensordot(np.tensordot(a,B,0), Ikij,2)

array([  0.,   4.,  -8.,   4.,   0.,   0.,   0., -38.])

In [83]:
np.tensordot(a, np.tensordot(B, Gkij,1),0)

ValueError: shape-mismatch for sum

In [84]:
np.einsum('i,j,kij->k',a, B, Gkij)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (8,)->(8,newaxis) (8,)->(8) (4,4,4)->(4,4,4) 

In [ ]:
vsr.EGA(*a) * vsr.EGA(*B)

In [ ]:
def rotor_estimation_ls_svd(points_a, points_b):
    
#     gp_table = np.array([1,  2,  3,  4,  5,  6,  7,  8,
#                          2,  1,  7, -6,  8, -4,  3,  5,
#                          3, -7,  1,  5,  4,  8, -2,  6,
#                          4,  6, -5,  1, -3,  2,  8,  7,
#                          5,  8, -4,  3, -1, -7,  6, -2,
#                          6,  4,  8, -2,  7, -1, -5, -3,
#                          7, -3,  2,  8, -6,  5, -1, -4, 
#                          8,  5,  6,  7, -2, -3, -4, -1]).reshape(8,8)

    M = np.zeros((8,8))
    mask = np.array([1,2,3,4,5,6,7,8])
    for i in range(8):
        W = np.zeros((8,8))
        for j in range(8):
            a = vsr.EGA(0,0,0,0,0,0,0,0)
            b = vsr.EGA(0,0,0,0,0,0,0,0)
            a[i] = 1.
            b[j] = 1.
            M[i,j] = np.dot(mask, np.array(a * b))
    gp_table = M.copy()

    def gp_tensor():
        dim = 8
        tensor = np.zeros((8,8,8))
        for k in range(dim):
            for i in range(dim):
                for j in range(dim):
                    val = gp_table[i,j]
                    if abs(val) == k + 1:
                        tensor[k,i,j] = np.sign(val)
        return tensor

    def gp_left_matrix(multivector):
        tensor = gp_tensor()
        matrix = np.zeros((8,8))
        for i in range(8):
            t = tensor[i,:,:]
            matrix[i,:] = np.inner(t.T,np.array(multivector).T).reshape(-1)
        return matrix

    def gp_right_matrix(multivector):
        tensor = gp_tensor()
        matrix = np.zeros((8,8))
        for i in range(8):
            t = tensor[i,:,:]
            matrix[i,:] = np.inner(np.array(multivector).T,t).reshape(-1)
        return matrix

#     A = [np.array([0.0, p[0], p[1], p[2], 0.0, 0.0, 0.0, 0.0]).reshape(8,1) for p in points_a]
#     B = [np.array([0.0, p[0], p[1], p[2], 0.0, 0.0, 0.0, 0.0]).reshape(8,1) for p in points_b]

    gp_a = np.row_stack([
        np.delete(np.delete(gp_right_matrix(a),[0,4,5,6],0), [1,2,3,7],1)
         for a in points_a])
    
    b_gp = np.row_stack([
            np.delete(np.delete(gp_left_matrix(b),[0,4,5,6],0), [1,2,3,7],1) for b in points_b])
    m = gp_a - b_gp

    [U,s,Vt] = np.linalg.svd(m)
    print(Vt.T)
    print(s)

    names = ('sc', 'e1', 'e2', 'e3', 'e12', 'e13', 'e23', 'e123')
    res = np.recarray(1, formats = 8*['f8'], names=names, buf=Vt.T[:,-2])
    rotor = np.array([res['sc'], res['e12'], res['e13'], res['e23']])
    return rotor, m

In [ ]:
print(points_a)
print(points_b)
r,m2 = rotor_estimation_ls_svd(points_a, points_b)
vsr.Rot(*r)

In [ ]:
print(rotor)

In [ ]:
gp_table = np.array([1,  2,  3,  4,  5,  6,  7,  8,
                     2,  1,  7, -6,  8, -4,  3,  5,
                     3, -7,  1,  5,  4,  8, -2,  6,
                     4,  6, -5,  1, -3,  2,  8,  7,
                     5,  8, -4,  3, -1, -7,  6, -2,
                     6,  4,  8, -2,  7, -1, -5, -3,
                     7, -3,  2,  8, -6,  5, -1, -4, 
                     8,  5,  6,  7, -2, -3, -4, -1]).T.reshape(8,8)
print(gp_table.T)

In [ ]:
M = np.zeros((8,8))
mask = np.array([1,2,3,4,5,6,7,8])
for i in range(8):
    W = np.zeros((8,8))
    for j in range(8):
        a = vsr.EGA(0,0,0,0,0,0,0,0)
        b = vsr.EGA(0,0,0,0,0,0,0,0)
        a[i] = 1.
        b[j] = 1.
        M[i,j] = np.dot(mask, np.array(a * b))
gp_table = M.T.copy()
print(gp_table.T)

In [ ]:
print(Multivector().vector(points_a[0]).gp_right_matrix())
print(Multivector().vector(points_b[0]).gp_left_matrix())
print(m2[:8])

In [ ]:
r = rotor
vsr.EGA(0,1,0,0,0,0,0,0).spin(vsr.EGA(r[0],0,0,0,r[1],r[2],r[3],0))

In [3]:
rotor = vsr.Biv(0,-pi/8,0).exp()
print(rotor)
n_points = 3
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3))
            for i in range(n_points)]
points_b = [point.spin(rotor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))) 
                  for point in points_b]

ega_a = [vsr.EGA(p) for p in points_a]
ega_b = [vsr.EGA(p) for p in points_b]

def matrix(a, b):
    m = np.zeros((8,8))
    for i in range(8):
        ei = vsr.EGA(0,0,0,0,0,0,0,0)
        ei[i] = i
        m[i,:] = np.array(ei * a - b * ei)
    return m


m = np.row_stack([
        np.delete(np.delete(matrix(a,b),[0,4,5,6],0), [1,2,3,7],1)
        for a, b in zip(ega_a, ega_b)]).copy()

U,s,Vt = np.linalg.svd(m)
print(Vt.T)
print(s)

Rot: [ 0.92 0 0.38 0 ]
[[-0.11 -0.28 -0.24 -0.92]
 [-0.43  0.68 -0.6  -0.  ]
 [ 0.27  0.68  0.57 -0.38]
 [ 0.85  0.09 -0.51 -0.  ]]
[ 8.53  5.3   3.53  0.  ]


In [ ]:
ega_a = [vsr.EGA(p) for p in points_a]
ega_b = [vsr.EGA(p) for p in points_b]

M = np.array([[1,0,0,0,0,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,1,0]])
print(M)

def matrix(a, b):
    m = np.zeros((8,8))
    for i in range(8):
        ei = vsr.EGA(0,0,0,0,0,0,0,0)
        ei[i] = i
        m[i,:] = np.array(ei * a - b * ei)
    return m


# m = np.row_stack([
#         np.delete(np.delete(matrix(a,b),[0,4,5,6],0), [1,2,3,7],1)
#         for a, b in zip(ega_a, ega_b)]).copy()

m = np.row_stack(np.dot(matrix(a,b),M.T) for a, b in zip(ega_a, ega_b))

U,s,Vt = np.linalg.svd(m)
# print(Vt.T)
print(vsr.Rot(*Vt[-1]))
# print(s)

In [52]:
matrix(ega_a[0], ega_b[0])

array([[  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ],
       [ -1.05,   0.  ,   0.  ,   0.  ,  -1.15,   2.55,   0.  ,   0.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   0.31,   0.  ,   5.09,   0.  ],
       [ -0.19,   0.  ,   0.  ,   0.  ,   0.  ,   0.47,   3.46,   0.  ],
       [  0.  ,  -4.61,   0.62,   0.  ,   0.  ,   0.  ,   0.  ,  -0.26],
       [  0.  ,  12.73,   0.  ,   0.78,   0.  ,   0.  ,   0.  ,  -0.  ],
       [  0.  ,   0.  ,  15.28,   6.92,   0.  ,   0.  ,   0.  ,  -6.33],
       [  0.  ,   0.  ,   0.  ,   0.  ,  -0.45,  -0.  ,  -7.38,   0.  ]])

In [54]:
np.delete(np.delete(matrix(ega_a[0],ega_b[0]),[0,4,5,6],0), [1,2,3,7],1)

array([[-1.05, -1.15,  2.55,  0.  ],
       [ 0.  ,  0.31,  0.  ,  5.09],
       [-0.19,  0.  ,  0.47,  3.46],
       [ 0.  , -0.45, -0.  , -7.38]])

In [55]:
r = np.array([1,2,3,4,5,6,7,8]).T

In [57]:
vsr.CGA(vsr.Mot(1,2,3,4,5,6,7,8))

CGA: [ 1 0 0 0 0 0 2 3 4 0 0 0 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 8 0 0 0 0 ]

In [59]:
np.delete(matrix(ega_a[0],ega_b[0]),[0,4,5,6],0)

array([[-1.05,  0.  ,  0.  ,  0.  , -1.15,  2.55,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.31,  0.  ,  5.09,  0.  ],
       [-0.19,  0.  ,  0.  ,  0.  ,  0.  ,  0.47,  3.46,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  , -0.45, -0.  , -7.38,  0.  ]])

In [60]:
motor

NameError: name 'motor' is not defined

In [ ]:
Mrotij = np.array([[1,0,0,0,0,0,0,0],
                   [0,0,0,0,1,0,0,0],
                   [0,0,0,0,0,1,0,0],
                   [0,0,0,0,0,0,1,0]])

In [ ]:
np.einsum('j,ij->i',r,Mrotij)

In [ ]:
cga_a = [vsr.CGA(p.null()) for p in points_a]
cga_b = [vsr.CGA(p.null()) for p in points_b]
def matrix(a, b):
    m = np.zeros((32,32))
    for i in range(32):
        ei = vsr.CGA(*np.zeros(32))
        ei[i] = i
        m[i,:] = np.array(ei * a - b * ei)
    return m
k = matrix(cga_a[0], cga_b[0])


m = np.row_stack([matrix(a,b) for a,b in zip(cga_a, cga_b)])
U,s,Vt = np.linalg.svd(m)
print(Vt.T[-1])

In [ ]:
import time
t1 = time.time()
vsr.CGA(vsr.Vec(1,2,3).null()).spin(vsr.CGA(motor))
t2 = time.time()
print(t2-t1)

In [92]:
t1 = time.time()
vsr.Vec(1,2,3).null().spin(motor)
t2 = time.time()
print(t2-t1)

NameError: name 'time' is not defined

In [151]:
np.set_printoptions(linewidth=200,precision=2)

In [30]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(1,1,1).unit() * np.pi/6.0)
print(motor)
n_points = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null()
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

cga_a = [vsr.CGA(p) for p in points_a]
cga_b = [vsr.CGA(p) for p in points_b]

def set_idx(idx):
    a = np.zeros(32)
    a[idx] = 1.
    return a

M = np.array([set_idx(0), 
              set_idx(6), set_idx(7), set_idx(8),
              set_idx(12), set_idx(13), set_idx(14), 
              set_idx(27)])


def matrix(a, b):
    m = np.zeros((32,32))
    for i in range(32):
        ei = vsr.CGA(*np.zeros(32))
        ei[i] = i
        m[i,:] = np.array(ei * a - b * ei)
    return np.dot(m,M.T)[1:6,:]
# print(matrix(cga_a[0],cga_b[0])[1:6,:] )

m = np.row_stack([matrix(a,b) for a, b in zip(cga_a, cga_b)]).copy()
# print(m)
U,s,Vt = np.linalg.svd(m)
print(Vt.T)
print(s)

Mot: [ 0.87 -0.29 -0.29 -0.29 -0.72 -0.43 -0.14 0.14 ]
[[ 0.18  0.14 -0.53  0.76 -0.27  0.1  -0.08  0.  ]
 [-0.    0.16 -0.07 -0.06 -0.11  0.26  0.94  0.  ]
 [ 0.17 -0.01 -0.21 -0.12 -0.22 -0.9   0.2   0.  ]
 [ 0.13 -0.2  -0.17  0.24  0.9  -0.15  0.18  0.  ]
 [-0.2  -0.13 -0.8  -0.51  0.04  0.19 -0.12  0.  ]
 [-0.01 -0.95  0.05  0.14 -0.25  0.07  0.12  0.  ]
 [-0.94  0.04  0.    0.26  0.03 -0.21  0.07  0.  ]
 [ 0.    0.    0.    0.    0.    0.   -0.    1.  ]]
[ 54.67  38.87  29.06  25.69  18.61  12.98   9.48   0.  ]


In [16]:
set_idx(1)

array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.])

In [ ]:
ega_a = [vsr.EGA(p) for p in points_a]
ega_b = [vsr.EGA(p) for p in points_b]

M = np.array([[1,0,0,0,0,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,1,0]])
print(M)

def matrix(a, b):
    m = np.zeros((8,8))
    for i in range(8):
        ei = vsr.EGA(0,0,0,0,0,0,0,0)
        ei[i] = i
        m[i,:] = np.array(ei * a - b * ei)
    return m


# m = np.row_stack([
#         np.delete(np.delete(matrix(a,b),[0,4,5,6],0), [1,2,3,7],1)
#         for a, b in zip(ega_a, ega_b)]).copy()

m = np.row_stack(np.dot(matrix(a,b),M.T) for a, b in zip(ega_a, ega_b))

U,s,Vt = np.linalg.svd(m)
# print(Vt.T)
print(vsr.Rot(*Vt[-1]))
# print(s)

In [125]:
e0  = vsr.CGA(vsr.Mot(1,0,0,0,0,0,0,0))  
e12 = vsr.CGA(vsr.Mot(0,1,0,0,0,0,0,0))
e13 = vsr.CGA(vsr.Mot(0,0,1,0,0,0,0,0))
e23 = vsr.CGA(vsr.Mot(0,0,0,1,0,0,0,0))
e1i = vsr.CGA(vsr.Mot(0,0,0,0,1,0,0,0))
e2i = vsr.CGA(vsr.Mot(0,0,0,0,0,1,0,0))
e3i = vsr.CGA(vsr.Mot(0,0,0,0,0,0,1,0))
e123i = vsr.CGA(vsr.Mot(0,0,0,0,0,0,0,1))

In [8]:
a = cga_a[0]
b = cga_b[0]
e0 * a - b * e0

NameError: name 'e0' is not defined

In [13]:
(lambda idx : [None for np.zeros(32); a[idx] = 1)(3)

SyntaxError: invalid syntax (<ipython-input-13-9d1c0e63f679>, line 1)

In [142]:
(e12 * a - b * e12)

CGA: [ 0 1.5 -2.6 0 0 0 0 0 0 0 0 0 0 0 0 0 -1.7 0 0 0 -3.1 0 0 0 0 0 0 0 0 0 0 0 ]

In [144]:
np.delete((e12 * a - b * e12),[0,6,7,8,9,10,11,12,13,14,15,26,27,28,29,30])

array([ 2., -3.,  0.,  0.,  0., -2.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0.,  0.])

In [132]:
e13 * a - b * e13

CGA: [ 0 1 0 -2.6 0 0 0 0 0 0 0 0 0 0 0 0 1.5 0 0 0 0 -3.1 0 0 0 0 0 0 0 0 0 0 ]

In [139]:
e123i * a - b * e123i

CGA: [ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -2 0 0 0 -1 1.5 -2.6 0 0 0 0 0 0 0 0 0 ]

In [18]:
vsr.CGA(vsr.Mot(1,2,3,4,5,6,7,8))

CGA: [ 1 0 0 0 0 0 2 3 4 0 0 0 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 8 0 0 0 0 ]

In [80]:
vsr.CGA(vsr.Mot(1,2,3,4,5,6,7,8))

CGA: [ 1 0 0 0 0 0 2 3 4 0 0 0 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 8 0 0 0 0 ]

In [122]:
vsr.Mot(vsr.CGA(vsr.Rot(1,2,3,4)) * vsr.CGA(vsr.Vec(1,2,3)))

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (Mot, float, float, float, float, float, float, float, float) -> None


In [ ]:
import scipy.linalg as linalg
U,s,Vh = linalg.svd(m)

In [ ]:
import scipy.io as io
io.savemat("/home/lars/m.mat", {"m":m})

In [ ]:
M = io.loadmat("/home/lars/Downloads/M.mat")["M"]
print(M)
U,s,Vt = np.linalg.svd(M)
print(s)

In [54]:
print(m[:8])

[[  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [ -0.39   0.     0.     0.     0.     0.    -3.49   0.95   0.     2.     0.
    0.     5.44   0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.    -5.4    0.     1.9    0.     4.
    0.     0.    10.88   0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  3.35   0.     0.     0.     0.     0.     0.    -8.09  10.47   0.     0.
    6.     0.     0.    16.32   0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.   -10.79
   13.96  -3.8    0.

In [ ]:
print(M[:8])

In [ ]:
matrix(vsr.EGA(1,0,0,0,0,0,0,0), vsr.EGA(0,0,0,0,0,0,0,0)).T

In [ ]:
print(vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(1,0,0,0,0,0,0,0))
print(vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(0,1,0,0,0,0,0,0))
print(vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(0,0,1,0,0,0,0,0))
print(vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(0,0,0,1,0,0,0,0))
print(vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(0,0,0,0,1,0,0,0))
print(vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(0,0,0,0,0,1,0,0))
print(vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(0,0,0,0,0,0,1,0))
print(vsr.EGA(0,1,0,0,0,0,0,0) * vsr.EGA(0,0,0,0,0,0,0,1))

In [ ]:
np.array([np.array(vsr.EGA(0,0,1,0,0,0,0,0) * vsr.EGA(1,0,0,0,0,0,0,0)),
          np.array(vsr.EGA(0,0,1,0,0,0,0,0) * vsr.EGA(0,1,0,0,0,0,0,0)),
          np.array(vsr.EGA(0,0,1,0,0,0,0,0) * vsr.EGA(0,0,1,0,0,0,0,0)),
          np.array(vsr.EGA(0,0,1,0,0,0,0,0) * vsr.EGA(0,0,0,1,0,0,0,0)),
          np.array(vsr.EGA(0,0,1,0,0,0,0,0) * vsr.EGA(0,0,0,0,1,0,0,0)),
          np.array(vsr.EGA(0,0,1,0,0,0,0,0) * vsr.EGA(0,0,0,0,0,1,0,0)),
          np.array(vsr.EGA(0,0,1,0,0,0,0,0) * vsr.EGA(0,0,0,0,0,0,1,0)),
          np.array(vsr.EGA(0,0,1,0,0,0,0,0) * vsr.EGA(0,0,0,0,0,0,0,1))]).T

In [ ]:
Multivector()._gp_tensor[2,:,:]

In [ ]:
vsr.EGA(0,0,0,0,0,0,0,0) * vsr.EGA(0,a[0],a[1],0,a[2],0,0,0)

In [ ]:
np.inner(matrix(vsr.EGA(vsr.Vec(1,2,3)), vsr.EGA(0,0,0,0,0,0,0,0)), vsr.EGA(vsr.Vec(-12,9,-13)))

In [ ]:
vsr.Vec(1,2,3) * vsr.Vec(-12,9,-13)

In [ ]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
a = vsr.CGA(motor)
print(a)

In [ ]:
a = vsr.EGA(1,0,0,0,0,0,0,0)
m2 = np.zeros((8,8))
for i in range(8):
    
    ei = vsr.EGA(*np.zeros(8))
    ei[i] = 1.0
    m2[:,i] = ei * vsr.EGA(1,0,0,0,0,0,0,0)
print(m)

In [ ]:
np.sum(m2,0)

In [ ]:
M = np.zeros((8,8))
for i in range(8):
    W = np.zeros((8,8))
    for j in range(8):
        a = vsr.EGA(0,0,0,0,0,0,0,0)
        b = vsr.EGA(0,0,0,0,0,0,0,0)
        a[i] = 1.
        b[j] = j + 1
        W[i,:] = np.array(a * b)
    print(np.sum(W,0))
    M[i,:] = np.sum(W,0)
print(M)

In [ ]:
M = np.zeros((8,8))
mask = np.array([1,2,3,4,5,6,7,8])
for i in range(8):
    W = np.zeros((8,8))
    for j in range(8):
        a = vsr.EGA(0,0,0,0,0,0,0,0)
        b = vsr.EGA(0,0,0,0,0,0,0,0)
        a[i] = 1.
        b[j] = 1.
        M[i,j] = np.dot(mask, np.array(a <= b))
print(M.T)

In [ ]:
def row(a):
    M = np.zeros(8)
    for i in range(8):
        b = vsr.EGA(0,0,0,0,0,0,0,0)
        b[i] = i + 1
        M += np.array(a * b)
    return M

for i in range(8):
    ei = vsr.EGA(0,0,0,0,0,0,0,0)
    ei[i] = 1.
    print(row(ei))
    

In [ ]:
np.dot([1,2,3,4,5,6,7,8], np.array(vsr.EGA(0,0,0,0,0,1,0,0) * vsr.EGA(0,0,0,0,0,0,0,1)))